##**필요한 라이브러리**

* urllib3 [HTTP 통신]
* json
* pandas
* time
* os
* IPython(HTML, display)
* ipywidgets

In [ ]:
import urllib3
import json
import pandas as pd
from time import sleep
import os
import time

In [ ]:
from IPython.display import HTML, display
import ipywidgets as widgets

언어 분석 기술(문어) =>"http://aiopen.etri.re.kr:8000/WiseNLU"


언어 분석 기술(구어) => "http://aiopen.etri.re.kr:8000/WiseNLU_spoken"

API 요청을 한번에 최대 1만자까지 가능하며 한 API Key당 하루 5000건까지 가능하다.

1인당 무조건 1key가 지급되기 때문에 Key를 추가하고 싶으면 명의가 필요하다.

In [ ]:
#지급받은 API Key
API_Key_list = ["3461b0be-c327-46df-afc4-7f5a4b70855f"]
accessKey = API_Key_list[0]

##**요청 분석 코드**

---


* 형태소 분석(문어/구어) : "morp"
* 어휘의미 분석(동음이의어)(문어) : "wsd"
* 어휘의미 분석(다의어)(문어) : "wsd_poly"
* 개체명 인식(문어/구어): "ner"
* 의존 구문 분석(문어) : "dparse"
* 의미역 인식(문어) : "srl"

In [ ]:
code = ["morp", "wsd", "wsd_poly", "ner", "dparse", "srl"]
code_kor = ["형태소 분석(문어/구어)", 
            "어휘의미 분석(동음이의어)(문어)", 
            "어휘의미 분석(다의어)(문어)", 
            "개체명 인식(문어/구어)", 
            "의존 구문 분석(문어)", 
            "의미역 인식(문어)"]

###**구글드라이브에 있는 kowiki_preprocessing.txt 파일 읽기**


1. 구글드라이브 마운트 시키기

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


2. 구글드라이브의 kowiki.txt 파일 불러오기

In [ ]:
input_path = '/content/gdrive/MyDrive/Storage/input'
input_files = sorted(os.listdir(input_path))
files_toggle = widgets.ToggleButtons(
    options = input_files,
    disable=False,
    button_style=''
)
print('분석할 파일 선택')
files_toggle

분석할 파일 선택


ToggleButtons(options=('kowiki.txt',), value='kowiki.txt')

In [ ]:
file_path=input_path + '/' + input_files[files_toggle.index]
df=pd.read_csv(file_path, error_bad_lines=False, header=None, names=["text"])
print(df)

                                                      text
0        지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 ...
1        1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하...
2        1976년 미합중국  제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으...
3        카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나...
4        그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 1979년 백악관에서...
...                                                    ...
2046267  예컨대 발라가 자본재를 노동과 토지에 버금하는 제3의 생산요소로 보고서 그 용역가격...
2046268  케인스에 의하면 이자는 순수한 화폐적 현상이며 실물자본의 생산력이나 투자의 한계효율...
2046269  여기서는 이자가 대부자금의 가격이며 이자율은 어느 기간의 자금수요의 흐름이 균형되는...
2046270  산업 분류는 비슷한 생산 프로세스, 유사한 제품, 또는 금융 시장 내의 비슷한 행동...
2046271  경제이론에 있어서 산업이라 함은 동일한 상품을 생산하는 기업의 집단이라고 할 수 있...

[2046272 rows x 1 columns]


3. API를 통해 받은 Json 파일을 저장할 위치 설정


In [ ]:
save_path='/content/gdrive/MyDrive/Storage/output/exobrain.txt'

4. 저장 공간에 이미 API를 받은 기록이 있는지 확인

In [ ]:
index = 0
if os.path.isfile(save_path): #파일이 이미 존재한다면 불러오기 기능 수행
  save_df = pd.read_csv(save_path, error_bad_lines=False, header=None, names=["exobrain_json"])
  index = len(save_df)
else:                         #파일이 없다면 DataFrame 새로 만들기
  save_df = pd.DataFrame(columns=["exobrain_json"])
print(save_df)

                                         exobrain_json
0    {"result":0,"return_object":{"doc_id":"","DCT"...
1    {"result":0,"return_object":{"doc_id":"","DCT"...
2    {"result":0,"return_object":{"doc_id":"","DCT"...
3    {"result":0,"return_object":{"doc_id":"","DCT"...
4    {"result":0,"return_object":{"doc_id":"","DCT"...
..                                                 ...
495  {"result":0,"return_object":{"doc_id":"","DCT"...
496  {"result":0,"return_object":{"doc_id":"","DCT"...
497  {"result":0,"return_object":{"doc_id":"","DCT"...
498  {"result":0,"return_object":{"doc_id":"","DCT"...
499  {"result":0,"return_object":{"doc_id":"","DCT"...

[500 rows x 1 columns]


##**response.status의 오류 발생 코드**
---
* 400 : Required arguments is empty
(필수 파라미터의 값이 없는 경우)
* 400 : One ore more arguments are not valid
(파라미터의 값이 유효하지 않은 경우)
* 413 : Request Entity Too Large
(요청 문장 또는 어휘의 크기가 서버가 처리할 수 있는 것보다 큰 경우)
* 429 : Too Many Requests 
(사용자가 주어진 시간 내에 서버에 너무 많은 요청을 하는 경우)
* 404 : Unknown Handler 
(등록되지 않은 서비스를 요청한 경우)
* 408 : Handler Timeout 
(서버의 요청 대기가 시간을 초과한 경우)
* 500 : ETRI service server connection refused 
(ETRI 분석 서버에서 요청을 받지 못하는 경우)
* 500 : ETRI service server is not exists 
(수행 가능한 ETRI 분석 서버가 없는 경우)
* 500 : Recipient Failure
(ETRI 분석 서버에서 요청을 처리하지 못하는 경우)
* 500 : Unknown ReplyFailure
(API 요청에 알 수 없는 내부 오류가 발생한 경우)
* 500 : Unknown Exception
(알 수 없는 내부 오류가 발생한 경우)

In [ ]:
Error_codes = ["400", "413", "429", "404", "408", "500"]
Error_messages = ["필수 파라미터의 값이 없거나 유효하지 않습니다.",
                  "문장 또는 어휘의 크기가 너무 큽니다.",
                  "서비스를 짧은 시간내에 많이 요청하셨습니다.",
                  "등록되지 않은 서비스 입니다.",
                  "서버의 요청 대기가 시간을 초과했습니다.",
                  "서버에서 요청을 처리할 수 없습니다."]

##**Exobrain API 서버로 kowiki text 분석 요청하기**

1. API 서버로 얼마나 작업을 수행할지, 무슨 작업을 할지 선택

In [ ]:
# 한번 작업을 돌리는 수 설정 (API 키당 최대 일간 횟수 : 5000)
slider = widgets.IntSlider(100, min=0, max= 5000)
print("한 Step Size 설정")
slider


한 Step Size 설정


IntSlider(value=100, max=5000)

In [ ]:
#형태소 분석일 경우 code[0]
radio = widgets.RadioButtons(
    options=code_kor,
    value=code_kor[0],
    description="",
    disabled=False
)
analysisCode = code[0]
print("분석 작업 선택")
radio

분석 작업 선택


RadioButtons(options=('형태소 분석(문어/구어)', '어휘의미 분석(동음이의어)(문어)', '어휘의미 분석(다의어)(문어)', '개체명 인식(문어/구어)', '의존 구문 분석(문어…

In [ ]:
analysis_index = radio.index
openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"

literary_style = ['문어']
if analysis_index in [0, 3]:
  literary_style.append('구어')

style_toggle = widgets.ToggleButtons(
    options = literary_style,
    description='문체선택',
    disable=False,
    button_style='',
    tooltips=['문어체 선택','구어체 선택']
)

style_toggle

ToggleButtons(description='문체선택', options=('문어', '구어'), tooltips=('문어체 선택', '구어체 선택'), value='문어')

2. 진행상황을 보기 위한 함수 설정

In [ ]:
def progress(value, max=100):
  return HTML("""
       <progress
           value='{value}'
           max='{max}',
           style='width: 100%'
      >
         {value}
      </progress>
  """.format(value=value, max=max))

3. Exobrain API서버로 request를 보내고 response를 받아 DataFrame에 저장한다.

In [ ]:
display_count = 1
step = slider.value
analysisCode = code[radio.index]
if style_toggle.index == 1:
  openApiURL += "_spoken"
out = display(progress(display_count,step), display_id=True)
for item in df["text"].iloc[index:step+index].iteritems():

  text = item[1]

  #request를 보내기 위한 변수값 설정
  requestJson = {
    "access_key" : accessKey,
    "argument" : {
        "text" : text,
        "analysis_code" : analysisCode
    }
  }

  #서버로 request를 보내 response값을 받아온다.
  http = urllib3.PoolManager()
  response = http.request(
    "POST",
    openApiURL,
    headers={"Content-Type" : "application/json; charset=UTF-8"},
    body=json.dumps(requestJson)
  )
  save_df.loc[index, 'exobrain_json'] = str(response.data, "utf-8")
  index += 1

  #진행상황을 알기 위한 diplay 코드
  out.update(progress(display_count, step))
  display_count += 1

  #에러 발생시 에러 문구 출력
  if str(response.status) in Error_codes:
    print("Error 발생 : "+ Error_messages[Error_codes.index(str(response.status))])
    print("Error 발생 위치 : " + index)
    print("Error 발생 문구 : " + df["text"].iloc[index])

  #과도하게 빠른 request를 보내는걸 방지 하기 위한 delay 설정
  sleep(0.3)

In [ ]:
print(save_df)

                                         exobrain_json
0    {"result":0,"return_object":{"doc_id":"","DCT"...
1    {"result":0,"return_object":{"doc_id":"","DCT"...
2    {"result":0,"return_object":{"doc_id":"","DCT"...
3    {"result":0,"return_object":{"doc_id":"","DCT"...
4    {"result":0,"return_object":{"doc_id":"","DCT"...
..                                                 ...
556  {"result":0,"return_object":{"doc_id":"","DCT"...
557  {"result":0,"return_object":{"doc_id":"","DCT"...
558  {"result":0,"return_object":{"doc_id":"","DCT"...
559  {"result":0,"return_object":{"doc_id":"","DCT"...
560  {"result":0,"return_object":{"doc_id":"","DCT"...

[561 rows x 1 columns]


In [ ]:
input_text = df["text"].iloc[0]
print(*input_text.split(". "), sep="\n")

지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다
조지아 공과대학교를 졸업하였다
그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다
1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다
그의 별명이 "땅콩 농부" 로 알려졌다.


In [ ]:
from pprint import pprint
output_text = save_df["exobrain_json"].iloc[0]
pprint(json.loads(output_text))

{'result': 0,
 'return_object': {'DCT': '',
                   'category': '',
                   'category_weight': 0.0,
                   'doc_id': '',
                   'entity': [],
                   'metaInfo': {},
                   'paragraphInfo': [],
                   'sentence': [{'NE': [],
                                 'NE_Link': [],
                                 'SRL': [],
                                 'WSD': [],
                                 'dependency': [],
                                 'id': 0.0,
                                 'morp': [{'id': 0.0,
                                           'lemma': '지미',
                                           'position': 0.0,
                                           'type': 'NNP',
                                           'weight': 0.0402366},
                                          {'id': 1.0,
                                           'lemma': '카터',
                                           'position': 7

오류 없이 API 값이 DataFrame에 저장되었다면 save_path의 위치로 저장하기

In [ ]:
save_df.to_csv(save_path, index=False, header=False)